In [ ]:
# ================================================
# STEP 1: Google Sheets Setup + Helper Functions
# ================================================

import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_formatting import set_row_heights
from datetime import datetime

# ------------------------------------------------
# CONSTANTS
# ------------------------------------------------
SPREADSHEET_NAME = "RealEstateListings"
SHEET_NAMES = [
    "all_listings",
    "kwsintmaarten",
    "sunshine",
    "ireteam",
    "trust",
    "century",
    "easyx",
    "cornerstone"
]

# Common column layout
HEADERS = [
    "Title", "Location", "Price", "Size", "Bedrooms",
    "Bathrooms", "Image", "Link", "Date n Time Extracted",
    "Property Type"
]

# ------------------------------------------------
# CONNECT TO EXISTING GOOGLE SHEET
# ------------------------------------------------
def setup_google_sheets():
    """Connects to an existing Google Sheet (RealEstateListings) and ensures all worksheets exist."""
    scope = [
        "https://www.googleapis.com/auth/spreadsheets",
        "https://www.googleapis.com/auth/drive"
    ]
    creds = ServiceAccountCredentials.from_json_keyfile_name("secret.json", scope)
    client = gspread.authorize(creds)

    try:
        spreadsheet = client.open(SPREADSHEET_NAME)
    except gspread.exceptions.SpreadsheetNotFound:
        raise Exception(
            f"❌ Spreadsheet '{SPREADSHEET_NAME}' not found. "
            f"Please create it manually in your Google Drive first."
        )

    # Ensure all required worksheets exist (but don’t create a new spreadsheet)
    for name in SHEET_NAMES:
        try:
            sheet = spreadsheet.worksheet(name)
        except gspread.exceptions.WorksheetNotFound:
            sheet = spreadsheet.add_worksheet(title=name, rows="1000", cols="20")
            sheet.append_row(HEADERS, value_input_option="USER_ENTERED")
    return spreadsheet


# ------------------------------------------------
# HELPER: Get existing (Title, Location, Price)
# ------------------------------------------------
def get_existing_records(sheet):
    """Loads existing records as a set of tuples (title, location, price)."""
    records = set()
    try:
        data = sheet.get_all_records()
        for row in data:
            key = (
                str(row.get("Title", "")).strip().lower(),
                str(row.get("Location", "")).strip().lower(),
                str(row.get("Price", "")).strip().lower()
            )
            records.add(key)
    except Exception as e:
        print(f"⚠️ Could not read existing records from {sheet.title}: {e}")
    return records


# ------------------------------------------------
# HELPER: Append Unique Rows
# ------------------------------------------------
def append_unique_rows(site_sheet, all_sheet, new_rows):
    """
    Adds only unique rows (no duplicates by Title+Location+Price)
    to both the site-specific sheet and the master all_listings sheet.
    """
    existing_site = get_existing_records(site_sheet)
    existing_all = get_existing_records(all_sheet)

    unique_rows = []
    for row in new_rows:
        key = (row[0].strip().lower(), row[1].strip().lower(), row[2].strip().lower())
        if key not in existing_site and key not in existing_all:
            unique_rows.append(row)
            existing_site.add(key)
            existing_all.add(key)

    if not unique_rows:
        print(f"✅ No new listings for {site_sheet.title}")
        return

    site_sheet.append_rows(unique_rows, value_input_option="USER_ENTERED")
    all_sheet.append_rows(unique_rows, value_input_option="USER_ENTERED")

    # Adjust image row heights
    total_rows = len(site_sheet.get_all_values())
    row_height_ranges = [(f"2:{total_rows}", 220)]
    set_row_heights(site_sheet, row_height_ranges)

    print(f"✅ Added {len(unique_rows)} new listings to {site_sheet.title} and all_listings.")


# ------------------------------------------------
# TEST / DEMO ENTRY POINT
# ------------------------------------------------
if __name__ == "__main__":
    spreadsheet = setup_google_sheets()
    print("✅ Connected to existing Google Sheet 'RealEstateListings'.")


✅ Connected to existing Google Sheet 'RealEstateListings'.


In [ ]:
#KWS Listings 1

import requests
from bs4 import BeautifulSoup
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime
from gspread_formatting import set_row_height
import time

# ----------------------------------
# Google Sheets Setup
# ----------------------------------
scope = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]

creds = ServiceAccountCredentials.from_json_keyfile_name('secret.json', scope)
client = gspread.authorize(creds)

SPREADSHEET_NAME = "RealEstateListings"
spreadsheet = client.open(SPREADSHEET_NAME)

def get_or_create_worksheet(name):
    try:
        return spreadsheet.worksheet(name)
    except gspread.exceptions.WorksheetNotFound:
        return spreadsheet.add_worksheet(title=name, rows=1000, cols=20)

sheet_all = get_or_create_worksheet("all_listings")
sheet_site = get_or_create_worksheet("kwsintmaarten")

HEADERS_ROW = [
    "Title", "Location", "Price", "Size", "Bedrooms", "Bathrooms",
    "Image", "Link", "Date n Time Extracted", "Property Type"
]

def ensure_headers(sheet):
    existing = sheet.row_values(1)
    if existing != HEADERS_ROW:
        sheet.clear()
        sheet.append_row(HEADERS_ROW, value_input_option='USER_ENTERED')
        set_row_height(sheet, "1:1", 40)

ensure_headers(sheet_all)
ensure_headers(sheet_site)

# ----------------------------------
# Fetch existing records to prevent duplicates
# ----------------------------------
def get_existing_records(sheet):
    records = sheet.get_all_records()
    existing = set()

    for r in records:
        # Safely get and clean values
        title = str(r.get("Title", "")).strip()
        location = str(r.get("Location", "")).strip()
        price = str(r.get("Price", "")).strip()

        # Skip blank or incomplete rows
        if not title or not location or not price:
            continue

        existing.add((title, location, price))

    return existing


# ----------------------------------
# Scrape Function
# ----------------------------------
def scrape_site(property_type, base_url):
    response = requests.get(base_url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    pagination = soup.find("ul", class_="pagination")
    total_pages = int(pagination.find_all("li")[-2].get_text(strip=True)) if pagination else 1

    all_rows = []
    print(f"Scraping {property_type} listings ({total_pages} pages)...")

    for page in range(1, total_pages + 1):
        url = f"{base_url}?page={page}"
        res = requests.get(url, headers=headers)
        if res.status_code != 200:
            print(f"⚠️ Failed page {page}")
            continue

        soup = BeautifulSoup(res.text, "html.parser")
        cards = soup.find_all("div", class_="card")

        for card in cards:
            title_tag = card.find("div", class_="card__heading")
            title = title_tag.get_text(strip=True) if title_tag else "-"
            if title == "-":
                continue

            location = card.find("div", class_="card__location")
            location_text = location.get_text(strip=True) if location else "-"

            price = card.find("div", class_="card__price")
            price_text = price.get_text(strip=True) if price else "-"
            if "|" in price_text:
                price_text = price_text.split("|")[1].strip().replace("USD", "").replace(".", "").strip()

            key = (title.strip(), location_text.strip(), price_text.strip())
            if key in existing_all or key in existing_site:
                continue  # skip duplicates

            options = card.find_all("div", class_="option__value")
            size = options[0].get_text(strip=True) if len(options) >= 1 else "-"
            bedrooms = options[1].get_text(strip=True) if len(options) >= 2 else "-"
            bathrooms = options[2].get_text(strip=True) if len(options) >= 3 else "-"

            img_tag = card.find("div", class_="card__image").find("img") if card.find("div", class_="card__image") else None
            image_url = img_tag["src"] if img_tag and img_tag.has_attr("src") else "-"
            if image_url != "-" and image_url.startswith("/"):
                image_url = "https://kwsintmaarten.com" + image_url
            image_formula = f'=IMAGE("{image_url}")' if image_url != "-" else "-"

            link_tag = card.find("a", href=True)
            link = "https://kwsintmaarten.com" + link_tag["href"] if link_tag else "-"

            date_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

            all_rows.append([
                title, location_text, price_text, size, bedrooms, bathrooms,
                image_formula, link, date_time, property_type
            ])

        print(f"✅ Page {page}/{total_pages} done ({len(cards)} listings)")

    print(f"Collected {len(all_rows)} new listings for {property_type}")
    return all_rows

# ----------------------------------
# Scrape and Batch Upload
# ----------------------------------
all_new_rows = []

for p_type, url in BASE_URLS.items():
    new_rows = scrape_site(p_type, url)
    all_new_rows.extend(new_rows)

if all_new_rows:
    # Batch write to both sheets in one go
    sheet_all.append_rows(all_new_rows, value_input_option='USER_ENTERED')
    sheet_site.append_rows(all_new_rows, value_input_option='USER_ENTERED')
    print(f"✅ Added {len(all_new_rows)} total new listings to Google Sheets!")
else:
    print("No new listings to add.")

print("🏁 Scraping complete without hitting Google Sheets quota.")


Scraping Sale listings (6 pages)...
✅ Page 1/6 done (24 listings)
✅ Page 2/6 done (24 listings)
✅ Page 3/6 done (24 listings)
✅ Page 4/6 done (24 listings)
✅ Page 5/6 done (24 listings)
✅ Page 6/6 done (24 listings)
Collected 120 new listings for Sale
Scraping Rent listings (4 pages)...
✅ Page 1/4 done (24 listings)
✅ Page 2/4 done (24 listings)
✅ Page 3/4 done (24 listings)
✅ Page 4/4 done (24 listings)
Collected 80 new listings for Rent
✅ Added 200 total new listings to Google Sheets!
🏁 Scraping complete without hitting Google Sheets quota.


In [ ]:
#Sunshine 2

import gspread
from oauth2client.service_account import ServiceAccountCredentials
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import time

# ----------------------------------
# Google Sheets Setup
# ----------------------------------
scope = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]
creds = ServiceAccountCredentials.from_json_keyfile_name("secret.json", scope)
client = gspread.authorize(creds)

SPREADSHEET_NAME = "RealEstateListings"
spreadsheet = client.open(SPREADSHEET_NAME)

def get_or_create_worksheet(name):
    for ws in spreadsheet.worksheets():
        if ws.title.strip().lower() == name.strip().lower():
            return ws
    return spreadsheet.add_worksheet(title=name, rows=2000, cols=20)

sheet_all = get_or_create_worksheet("all_listings")
sheet_site = get_or_create_worksheet("Sunshine")

HEADERS = [
    "Title", "Location", "Price", "Size", "Bedrooms", "Bathrooms",
    "Image", "Link", "Date n Time Extracted", "Property Type"
]

def ensure_headers(sheet):
    existing = sheet.row_values(1)
    if existing != HEADERS:
        sheet.clear()
        sheet.append_row(HEADERS, value_input_option='USER_ENTERED')

ensure_headers(sheet_all)
ensure_headers(sheet_site)

# ----------------------------------
# Duplicate Prevention
# ----------------------------------
def get_existing_records(sheet):
    records = sheet.get_all_records()
    existing = set()
    for r in records:
        title = str(r.get("Title", "")).strip()
        location = str(r.get("Location", "")).strip()
        price = str(r.get("Price", "")).strip()
        if title and location and price:
            existing.add((title, location, price))
    return existing

existing_all = get_existing_records(sheet_all)
existing_site = get_existing_records(sheet_site)

# ----------------------------------
# Scraping Config
# ----------------------------------
BASE_URLS = {
    "Sale": "https://www.sunshine-properties.com/en/st-maarten-real-estate-for-sale",
    "Rent": "https://www.sunshine-properties.com/en/st-maarten-long-term-rentals"
}

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                  "(KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}

# ----------------------------------
# Scraper Function with Pagination
# ----------------------------------
def scrape_sunshine(property_type, base_url):
    print(f"🔎 Scraping {property_type} listings from {base_url}")
    page = 1
    all_rows = []

    while True:
        url = f"{base_url}?page={page}"
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            print(f"❌ Failed to fetch page {page} (status {response.status_code})")
            break

        soup = BeautifulSoup(response.text, "html.parser")
        cards = soup.select("div > article.infos")
        if not cards:
            print(f"⚠️ No more listings found on page {page}. Ending pagination.")
            break

        for card in cards:
            # --- Title ---
            title_tag = card.select_one("h2")
            title = title_tag.get_text(strip=True) if title_tag else "No Title"

            # --- Location ---
            loc_tag = card.select_one("h3")
            location = loc_tag.get_text(strip=True) if loc_tag else "-"

            # --- Price ---
            price_tag = card.select_one("li.price")
            price = price_tag.get_text(strip=True).replace("$", "").replace(",", "").strip() if price_tag else "-"

            # --- Bedrooms / Bathrooms / Area ---
            bedrooms, bathrooms, size = "-", "-", "-"
            for li in card.select("ul li"):
                text = li.get_text(strip=True)
                if "bedroom" in text.lower():
                    bedrooms = text.lower().replace("bedrooms", "").replace("bedroom", "").strip()
                elif "bathroom" in text.lower():
                    bathrooms = text.lower().replace("bathrooms", "").replace("bathroom", "").strip()
                elif "m²" in text or "area" in text.lower():
                    size = text.strip()

            # --- Image ---
            figure_tag = card.find_previous_sibling("figure")
            img_tag = figure_tag.select_one("img") if figure_tag else None
            img_url = img_tag["src"] if img_tag and img_tag.has_attr("src") else "-"
            if img_url.startswith("/"):
                img_url = "https://www.sunshine-properties.com" + img_url
            image_formula = f'=IMAGE("{img_url}")' if img_url != "-" else "-"

            # --- Link ---
            link_tag = figure_tag.select_one("a") if figure_tag else None
            link = "https://www.sunshine-properties.com" + link_tag["href"] if link_tag else "-"

            # --- Date & Property Type ---
            date_extracted = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

            key = (title, location, price)
            if key in existing_all or key in existing_site:
                continue

            row = [title, location, price, size, bedrooms, bathrooms,
                   image_formula, link, date_extracted, property_type]
            all_rows.append(row)

        print(f"✅ Page {page} scraped ({len(cards)} listings)")
        page += 1
        time.sleep(2)  # polite delay

    print(f"✅ Found {len(all_rows)} new {property_type} listings in total.")
    return all_rows

# ----------------------------------
# Run Scraper
# ----------------------------------
all_new_rows = []
for prop_type, url in BASE_URLS.items():
    new_rows = scrape_sunshine(prop_type, url)
    all_new_rows.extend(new_rows)
    time.sleep(2)  # polite delay

if all_new_rows:
    sheet_all.append_rows(all_new_rows, value_input_option='USER_ENTERED')
    sheet_site.append_rows(all_new_rows, value_input_option='USER_ENTERED')
    print(f"✅ Added {len(all_new_rows)} new Sunshine listings to Google Sheets.")
else:
    print("⚠️ No new listings found. Check website structure or update selectors.")

print("🏁 Done! Data saved safely in 'RealEstateListings'.")


🔎 Scraping Sale listings from https://www.sunshine-properties.com/en/st-maarten-real-estate-for-sale
✅ Page 1 scraped (12 listings)
✅ Page 2 scraped (12 listings)
✅ Page 3 scraped (12 listings)
✅ Page 4 scraped (12 listings)
✅ Page 5 scraped (12 listings)
✅ Page 6 scraped (12 listings)
✅ Page 7 scraped (12 listings)
✅ Page 8 scraped (12 listings)
✅ Page 9 scraped (12 listings)
✅ Page 10 scraped (12 listings)
✅ Page 11 scraped (12 listings)
✅ Page 12 scraped (8 listings)
⚠️ No more listings found on page 13. Ending pagination.
✅ Found 128 new Sale listings in total.
🔎 Scraping Rent listings from https://www.sunshine-properties.com/en/st-maarten-long-term-rentals
✅ Page 1 scraped (12 listings)
✅ Page 2 scraped (12 listings)
✅ Page 3 scraped (12 listings)
✅ Page 4 scraped (3 listings)
⚠️ No more listings found on page 5. Ending pagination.
✅ Found 27 new Rent listings in total.
✅ Added 155 new Sunshine listings to Google Sheets.
🏁 Done! Data saved safely in 'RealEstateListings'.


In [ ]:
#Trust 3

import requests
from bs4 import BeautifulSoup
import time
from datetime import datetime
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_formatting import set_row_heights

# ----------------------------------
# Google Sheets Setup
# ----------------------------------
scope = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]
creds = ServiceAccountCredentials.from_json_keyfile_name("secret.json", scope)
client = gspread.authorize(creds)

SPREADSHEET_NAME = "RealEstateListings"
spreadsheet = client.open(SPREADSHEET_NAME)

def get_or_create_worksheet(name):
    for ws in spreadsheet.worksheets():
        if ws.title.strip().lower() == name.strip().lower():
            return ws
    return spreadsheet.add_worksheet(title=name, rows=2000, cols=20)

sheet_all = get_or_create_worksheet("all_listings")
sheet_site = get_or_create_worksheet("trust")

HEADERS = [
    "Title", "Location", "Price", "Size", "Bedrooms", "Bathrooms",
    "Image", "Link", "Date n Time Extracted", "Property Type"
]

def ensure_headers(sheet):
    existing = sheet.row_values(1)
    if existing != HEADERS:
        sheet.clear()
        sheet.append_row(HEADERS, value_input_option='USER_ENTERED')

ensure_headers(sheet_all)
ensure_headers(sheet_site)

# ----------------------------------
# Duplicate Prevention
# ----------------------------------
def get_existing_records(sheet):
    records = sheet.get_all_records()
    existing = set()
    for r in records:
        title = str(r.get("Title", "")).strip()
        location = str(r.get("Location", "")).strip()
        price = str(r.get("Price", "")).strip()
        if title and location and price:
            existing.add((title, location, price))
    return existing

existing_all = get_existing_records(sheet_all)
existing_site = get_existing_records(sheet_site)

# ----------------------------------
# Scraper Config
# ----------------------------------
BASE_URLS = {
    "Sale": "https://trustrealestatesxm.com/our-listings/houses-sale-st-maarten/",
    "Rent": "https://trustrealestatesxm.com/our-listings/long-term-rental/"
}

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

# ----------------------------------
# Scraper Function with Pagination
# ----------------------------------
def scrape_trust(property_type, base_url, max_pages=10):
    print(f"🔎 Scraping {property_type} listings from {base_url}")
    page = 1
    all_rows = []

    while page <= max_pages:
        url = f"{base_url}?page={page}"
        response = requests.get(url, headers=headers, timeout=20)
        if response.status_code != 200:
            print(f"❌ Failed to fetch page {page}")
            break

        soup = BeautifulSoup(response.text, "html.parser")
        listings = soup.find_all("div", class_="wrapper")
        if not listings:
            print(f"⚠️ No listings found on page {page}. Ending pagination.")
            break

        for listing in listings:
            # --- Title & Location ---
            title_tag = listing.find("div", class_="Listing-area")
            if title_tag:
                title = title_tag.find("p").get_text(strip=True)
                location = title_tag.find_all("p")[1].get_text(strip=True)
            else:
                title, location = "-", "-"

            # --- Link ---
            link_tag = listing.find("a", href=True)
            link = f"https://trustrealestatesxm.com{link_tag['href']}" if link_tag else "-"

            # --- Image ---
            img_div = listing.find("div", class_="Listing-photo")
            img_url = "-"
            if img_div:
                style = img_div.get("style", "")
                if "url(" in style:
                    img_url = style.split("url(")[1].split(")")[0].strip()
            img_formula = f'=IMAGE("{img_url}", 4, 300, 200)' if img_url != "-" else "-"

            # --- Price, Bedrooms, Bathrooms, Size ---
            price_tag = listing.find("div", class_="Listing-price")
            price = price_tag.get_text(" ", strip=True) if price_tag else "-"

            bathrooms = "-"
            bedrooms = "-"
            size = "-"
            table = listing.find("div", class_="Listing-content")
            if table:
                rows_table = table.find_all("tr")
                if len(rows_table) >= 2:
                    cols = rows_table[1].find_all("td")
                    if len(cols) >= 2:
                        bathrooms = cols[0].get_text(strip=True)
                        bedrooms = cols[1].get_text(strip=True)
                # Optionally parse size if available

            date_extracted = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

            # --- Duplicate check ---
            key = (title, location, price)
            if key in existing_all or key in existing_site:
                continue

            row = [title, location, price, size, bedrooms, bathrooms,
                   img_formula, link, date_extracted, property_type]
            all_rows.append(row)

        print(f"✅ Page {page} scraped ({len(listings)} listings)")
        page += 1
        time.sleep(1)

    print(f"✅ Found {len(all_rows)} new {property_type} listings in total.")
    return all_rows

# ----------------------------------
# Run Scraper
# ----------------------------------
all_new_rows = []
for prop_type, url in BASE_URLS.items():
    new_rows = scrape_trust(prop_type, url, max_pages=10)
    all_new_rows.extend(new_rows)
    time.sleep(1)

if all_new_rows:
    sheet_all.append_rows(all_new_rows, value_input_option='USER_ENTERED')
    sheet_site.append_rows(all_new_rows, value_input_option='USER_ENTERED')

    # Adjust row heights for images
    row_height_ranges = [(f"{i}:{i}", 220) for i in range(2, len(all_new_rows) + 2)]
    set_row_heights(sheet_site, row_height_ranges)

    print(f"✅ Added {len(all_new_rows)} new Trust listings to Google Sheets.")
else:
    print("⚠️ No new listings found.")

print("🏁 Done! Data saved safely in 'RealEstateListings'.")


🔎 Scraping Sale listings from https://trustrealestatesxm.com/our-listings/houses-sale-st-maarten/
✅ Page 1 scraped (16 listings)
✅ Page 2 scraped (16 listings)
✅ Page 3 scraped (16 listings)
✅ Page 4 scraped (16 listings)
✅ Page 5 scraped (16 listings)
✅ Page 6 scraped (6 listings)
✅ Page 7 scraped (6 listings)
✅ Page 8 scraped (6 listings)
✅ Page 9 scraped (6 listings)
✅ Page 10 scraped (6 listings)
✅ Found 110 new Sale listings in total.
🔎 Scraping Rent listings from https://trustrealestatesxm.com/our-listings/long-term-rental/
✅ Page 1 scraped (16 listings)
✅ Page 2 scraped (16 listings)
✅ Page 3 scraped (16 listings)
✅ Page 4 scraped (16 listings)
✅ Page 5 scraped (14 listings)
✅ Page 6 scraped (14 listings)
✅ Page 7 scraped (14 listings)
✅ Page 8 scraped (14 listings)
✅ Page 9 scraped (14 listings)
✅ Page 10 scraped (14 listings)
✅ Found 148 new Rent listings in total.
✅ Added 258 new Trust listings to Google Sheets.
🏁 Done! Data saved safely in 'RealEstateListings'.


In [ ]:
# easyx 4

import requests
from bs4 import BeautifulSoup
import time
from datetime import datetime
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_formatting import set_row_heights

# ----------------------------------
# Google Sheets Setup
# ----------------------------------
scope = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]
creds = ServiceAccountCredentials.from_json_keyfile_name("secret.json", scope)
client = gspread.authorize(creds)

SPREADSHEET_NAME = "RealEstateListings"
spreadsheet = client.open(SPREADSHEET_NAME)

def get_or_create_worksheet(name):
    for ws in spreadsheet.worksheets():
        if ws.title.strip().lower() == name.strip().lower():
            return ws
    return spreadsheet.add_worksheet(title=name, rows=2000, cols=20)

sheet_all = get_or_create_worksheet("all_listings")
sheet_site = get_or_create_worksheet("easyx")

HEADERS = [
    "Title", "Location", "Price", "Size", "Bedrooms", "Bathrooms",
    "Image", "Link", "Date n Time Extracted", "Property Type"
]

def ensure_headers(sheet):
    existing = sheet.row_values(1)
    if existing != HEADERS:
        sheet.clear()
        sheet.append_row(HEADERS, value_input_option='USER_ENTERED')

ensure_headers(sheet_all)
ensure_headers(sheet_site)

# ----------------------------------
# Duplicate Prevention
# ----------------------------------
def get_existing_records(sheet):
    records = sheet.get_all_records()
    existing = set()
    for r in records:
        title = str(r.get("Title", "")).strip()
        location = str(r.get("Location", "")).strip()
        price = str(r.get("Price", "")).strip()
        if title and location and price:
            existing.add((title, location, price))
    return existing

existing_all = get_existing_records(sheet_all)
existing_site = get_existing_records(sheet_site)

# ----------------------------------
# Scraper Config
# ----------------------------------
BASE_URLS = {
    "Sale": [
        "https://easysxm.com/property-type/land/",
        "https://easysxm.com/property-type/villas-and-condos/",
        "https://easysxm.com/property-type/commercial-sales/"
    ],
    "Rent": [
        "https://easysxm.com/property-type/long-term/"
    ]
}

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

# ----------------------------------
# Scraper Function with Pagination
# ----------------------------------
def scrape_easyx(property_type, base_urls, max_pages=10):
    all_rows = []
    for base_url in base_urls:
        print(f"🔎 Scraping {property_type} listings from {base_url}")
        page = 1

        while page <= max_pages:
            url = f"{base_url}page/{page}/"
            response = requests.get(url, headers=headers, timeout=20)
            if response.status_code != 200:
                print(f"❌ Failed to fetch page {page}")
                break

            soup = BeautifulSoup(response.text, "html.parser")
            listings = soup.find_all("div", class_="property-inner")
            if not listings:
                print(f"⚠️ No listings found on page {page}. Ending pagination.")
                break

            for listing in listings:
                # --- Title & Link ---
                title_tag = listing.select_one(".property-title a")
                title = title_tag.get_text(strip=True) if title_tag else "-"
                link = title_tag["href"] if title_tag else "-"

                # --- Location ---
                loc_tag = listing.select_one(".property-location span")
                location = loc_tag.get_text(strip=True) if loc_tag else "-"

                # --- Price ---
                price_tag = listing.select_one(".property-price")
                price = price_tag.get_text(strip=True) if price_tag else "-"

                # --- Bedrooms & Size ---
                bedrooms = "-"
                size = "-"
                bathrooms = "-"
                # EasySXM often has no explicit bedrooms/bathrooms/size fields, can be extended if needed

                # --- Image ---
                img_tag = listing.select_one(".property-image img")
                img_url = img_tag["src"] if img_tag else "-"
                img_formula = f'=IMAGE("{img_url}",4,300,200)' if img_url != "-" else "-"

                date_extracted = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

                # --- Duplicate check ---
                key = (title, location, price)
                if key in existing_all or key in existing_site:
                    continue

                row = [title, location, price, size, bedrooms, bathrooms,
                       img_formula, link, date_extracted, property_type]
                all_rows.append(row)

            print(f"✅ Page {page} scraped ({len(listings)} listings)")
            page += 1
            time.sleep(1)

    print(f"✅ Found {len(all_rows)} new {property_type} listings in total.")
    return all_rows

# ----------------------------------
# Run Scraper
# ----------------------------------
all_new_rows = []

for prop_type, urls in BASE_URLS.items():
    new_rows = scrape_easyx(prop_type, urls, max_pages=10)
    all_new_rows.extend(new_rows)
    time.sleep(1)

if all_new_rows:
    sheet_all.append_rows(all_new_rows, value_input_option='USER_ENTERED')
    sheet_site.append_rows(all_new_rows, value_input_option='USER_ENTERED')

    # Adjust row heights for images
    row_height_ranges = [(f"{i}:{i}", 220) for i in range(2, len(all_new_rows) + 2)]
    set_row_heights(sheet_site, row_height_ranges)

    print(f"✅ Added {len(all_new_rows)} new EasySXM listings to Google Sheets.")
else:
    print("⚠️ No new listings found.")

print("🏁 Done! Data saved safely in 'RealEstateListings'.")


🔎 Scraping Sale listings from https://easysxm.com/property-type/land/
✅ Page 1 scraped (15 listings)
✅ Page 2 scraped (15 listings)
✅ Page 3 scraped (15 listings)
✅ Page 4 scraped (15 listings)
✅ Page 5 scraped (9 listings)
❌ Failed to fetch page 6
🔎 Scraping Sale listings from https://easysxm.com/property-type/villas-and-condos/
✅ Page 1 scraped (15 listings)
✅ Page 2 scraped (15 listings)
✅ Page 3 scraped (15 listings)
✅ Page 4 scraped (15 listings)
✅ Page 5 scraped (15 listings)
✅ Page 6 scraped (15 listings)
✅ Page 7 scraped (15 listings)
✅ Page 8 scraped (15 listings)
✅ Page 9 scraped (15 listings)
✅ Page 10 scraped (15 listings)
🔎 Scraping Sale listings from https://easysxm.com/property-type/commercial-sales/
✅ Page 1 scraped (15 listings)
✅ Page 2 scraped (2 listings)
❌ Failed to fetch page 3
✅ Found 236 new Sale listings in total.
🔎 Scraping Rent listings from https://easysxm.com/property-type/long-term/
✅ Page 1 scraped (15 listings)
✅ Page 2 scraped (15 listings)
✅ Page 3 scr

In [ ]:
# Century 5

import requests
from bs4 import BeautifulSoup
import time
from datetime import datetime
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_formatting import set_row_heights

# ----------------------------------
# Google Sheets Setup
# ----------------------------------
scope = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]
creds = ServiceAccountCredentials.from_json_keyfile_name("secret.json", scope)
client = gspread.authorize(creds)

SPREADSHEET_NAME = "RealEstateListings"
spreadsheet = client.open(SPREADSHEET_NAME)

def get_or_create_worksheet(name):
    for ws in spreadsheet.worksheets():
        if ws.title.strip().lower() == name.strip().lower():
            return ws
    return spreadsheet.add_worksheet(title=name, rows=2000, cols=20)

sheet_all = get_or_create_worksheet("all_listings")
sheet_site = get_or_create_worksheet("century")

HEADERS = [
    "Title", "Location", "Price", "Size", "Bedrooms", "Bathrooms",
    "Image", "Link", "Date n Time Extracted", "Property Type"
]

def ensure_headers(sheet):
    existing = sheet.row_values(1)
    if existing != HEADERS:
        sheet.clear()
        sheet.append_row(HEADERS, value_input_option='USER_ENTERED')

ensure_headers(sheet_all)
ensure_headers(sheet_site)

# ----------------------------------
# Duplicate Prevention
# ----------------------------------
def get_existing_records(sheet):
    records = sheet.get_all_records()
    existing = set()
    for r in records:
        title = str(r.get("Title", "")).strip()
        location = str(r.get("Location", "")).strip()
        price = str(r.get("Price", "")).strip()
        if title and location and price:
            existing.add((title, location, price))
    return existing

existing_all = get_existing_records(sheet_all)
existing_site = get_existing_records(sheet_site)

# ----------------------------------
# Scraper Config
# ----------------------------------
BASE_URLS = {
    "Sale": [
        "https://www.century21-stmaarten.com/properties/?description=&property_type%5B%5D=1387&contract_type%5B%5D=2687",
        "https://www.century21-stmaarten.com/properties/?description=&property_type%5B%5D=47&contract_type%5B%5D=2687",
        "https://www.century21-stmaarten.com/properties/?contract_type%5B%5D=2687&property_type%5B%5D=870",
        "https://www.century21-stmaarten.com/property_type/sxm-land-for-sale/",
        "https://www.century21-stmaarten.com/property_type/commercial-real-estate-for-rent/"
    ],
    "Rent": [
        "https://www.century21-stmaarten.com/property_contract/long-term-rentals/"
    ]
}

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

# ----------------------------------
# Scraper Function with Pagination
# ----------------------------------
def scrape_century(property_type, base_urls, max_pages=10):
    all_rows = []
    for base_url in base_urls:
        print(f"🔎 Scraping {property_type} listings from {base_url}")
        page = 1

        while page <= max_pages:
            url = f"{base_url}?page={page}"
            response = requests.get(url, headers=headers, timeout=20)
            if response.status_code != 200:
                print(f"❌ Failed to fetch page {page}")
                break

            soup = BeautifulSoup(response.text, "html.parser")
            listings = soup.find_all("div", class_="row")
            if not listings:
                print(f"⚠️ No listings found on page {page}. Ending pagination.")
                break

            for listing in listings:
                title_tag = listing.select_one(".proerty-listing_title a")
                if not title_tag:
                    continue
                title = title_tag.get_text(strip=True)
                link = title_tag["href"]

                loc_tag = listing.select_one(".property-lisitng-location h6")
                location = loc_tag.get_text(strip=True) if loc_tag else "-"

                price_tag = listing.select_one(".price_rooms-sec strong")
                price = price_tag.get_text(strip=True) if price_tag else "-"

                bedrooms = "-"
                size = "-"
                bathrooms = "-"
                details = listing.select(".property-list-detail_content")
                for d in details:
                    p_tag = d.find("p")
                    h4_tag = d.find("h4")
                    if not p_tag or not h4_tag:
                        continue
                    label = p_tag.get_text(strip=True).lower()
                    value = h4_tag.get_text(strip=True)
                    if "bedroom" in label:
                        bedrooms = value
                    elif "size" in label or "area" in label:
                        size = value
                    elif "bathroom" in label:
                        bathrooms = value

                img_tag = listing.select_one(".property-main-img")
                img_url = img_tag["src"] if img_tag else "-"
                img_formula = f'=IMAGE("{img_url}",4,300,200)' if img_url != "-" else "-"

                date_extracted = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

                # --- Duplicate check ---
                key = (title, location, price)
                if key in existing_all or key in existing_site:
                    continue

                row = [title, location, price, size, bedrooms, bathrooms,
                       img_formula, link, date_extracted, property_type]
                all_rows.append(row)

            print(f"✅ Page {page} scraped ({len(listings)} listings)")
            page += 1
            time.sleep(1)

    print(f"✅ Found {len(all_rows)} new {property_type} listings in total.")
    return all_rows

# ----------------------------------
# Run Scraper
# ----------------------------------
all_new_rows = []

for prop_type, urls in BASE_URLS.items():
    new_rows = scrape_century(prop_type, urls, max_pages=10)
    all_new_rows.extend(new_rows)
    time.sleep(1)

if all_new_rows:
    sheet_all.append_rows(all_new_rows, value_input_option='USER_ENTERED')
    sheet_site.append_rows(all_new_rows, value_input_option='USER_ENTERED')

    # Adjust row heights for images
    row_height_ranges = [(f"{i}:{i}", 220) for i in range(2, len(all_new_rows) + 2)]
    set_row_heights(sheet_site, row_height_ranges)

    print(f"✅ Added {len(all_new_rows)} new Century21 listings to Google Sheets.")
else:
    print("⚠️ No new listings found.")

print("🏁 Done! Data saved safely in 'RealEstateListings'.")


🔎 Scraping Sale listings from https://www.century21-stmaarten.com/properties/?description=&property_type%5B%5D=1387&contract_type%5B%5D=2687
✅ Page 1 scraped (111 listings)
✅ Page 2 scraped (111 listings)
✅ Page 3 scraped (111 listings)
✅ Page 4 scraped (111 listings)
✅ Page 5 scraped (111 listings)
✅ Page 6 scraped (111 listings)
✅ Page 7 scraped (111 listings)
✅ Page 8 scraped (111 listings)
✅ Page 9 scraped (111 listings)
✅ Page 10 scraped (111 listings)
🔎 Scraping Sale listings from https://www.century21-stmaarten.com/properties/?description=&property_type%5B%5D=47&contract_type%5B%5D=2687
✅ Page 1 scraped (13 listings)
✅ Page 2 scraped (13 listings)
✅ Page 3 scraped (13 listings)
✅ Page 4 scraped (13 listings)
✅ Page 5 scraped (13 listings)
✅ Page 6 scraped (13 listings)
✅ Page 7 scraped (13 listings)
✅ Page 8 scraped (13 listings)
✅ Page 9 scraped (13 listings)
✅ Page 10 scraped (13 listings)
🔎 Scraping Sale listings from https://www.century21-stmaarten.com/properties/?contract_t

In [ ]:
# Cornerstone 6

import requests
from bs4 import BeautifulSoup
import time
from datetime import datetime
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_formatting import set_row_heights

# ----------------------------
# Google Sheets setup
# ----------------------------
scope = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
]

creds = ServiceAccountCredentials.from_json_keyfile_name('secret.json', scope)
client = gspread.authorize(creds)

SPREADSHEET_NAME = "RealEstateListings"
spreadsheet = client.open(SPREADSHEET_NAME)

def get_or_create_worksheet(name):
    for ws in spreadsheet.worksheets():
        if ws.title.strip().lower() == name.strip().lower():
            return ws
    return spreadsheet.add_worksheet(title=name, rows=2000, cols=20)

sheet_all = get_or_create_worksheet("all_listings")
sheet_site = get_or_create_worksheet("cornerstone")

HEADERS = [
    "Title", "Location", "Price", "Size", "Bedrooms", "Bathrooms",
    "Image", "Link", "Date n Time Extracted", "Property Type"
]

def ensure_headers(sheet):
    existing = sheet.row_values(1)
    if existing != HEADERS:
        sheet.clear()
        sheet.append_row(HEADERS, value_input_option='USER_ENTERED')

ensure_headers(sheet_all)
ensure_headers(sheet_site)

# ----------------------------
# Duplicate Prevention
# ----------------------------
def get_existing_records(sheet):
    records = sheet.get_all_records()
    existing = set()
    for r in records:
        title = str(r.get("Title", "")).strip()
        location = str(r.get("Location", "")).strip()
        price = str(r.get("Price", "")).strip()
        if title and location and price:
            existing.add((title, location, price))
    return existing

existing_all = get_existing_records(sheet_all)
existing_site = get_existing_records(sheet_site)

# ----------------------------
# Scraper Config
# ----------------------------
BASE_URLS = {
    "Sale": ["https://cornerstonerealestatesxm.com/property-type/for-sale/"],
    "Rent": ["https://cornerstonerealestatesxm.com/property-type/for-rent/"]
}

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

# ----------------------------
# Scraper Function
# ----------------------------
def scrape_cornerstone(property_type, base_urls, max_pages=10):
    all_rows = []
    for base_url in base_urls:
        print(f"🔎 Scraping {property_type} listings from {base_url}")
        page = 1

        while page <= max_pages:
            url = f"{base_url}?page={page}"
            response = requests.get(url, headers=headers, timeout=20)
            if response.status_code != 200:
                print(f"❌ Failed to fetch page {page}")
                break

            soup = BeautifulSoup(response.text, "html.parser")
            items = soup.select(".item-wrap")
            if not items:
                print(f"⚠️ No listings found on page {page}. Ending pagination.")
                break

            for item in items:
                # --- Title & Link ---
                title_tag = item.select_one(".item-title a")
                title = title_tag.text.strip() if title_tag else "-"
                link = title_tag["href"] if title_tag else "-"

                # --- Location ---
                loc_tag = item.select_one(".h-location .hz-figure")
                location = loc_tag.text.strip() if loc_tag else "-"

                # --- Price ---
                price_tag = item.select_one(".price")
                price = price_tag.text.strip() if price_tag else "-"

                # --- Bedrooms / Bathrooms / Size ---
                beds_tag = item.select_one(".h-beds .hz-figure")
                bedrooms = beds_tag.text.strip() if beds_tag else "-"
                baths_tag = item.select_one(".h-baths .hz-figure")
                bathrooms = baths_tag.text.strip() if baths_tag else "-"
                area_tag = item.select_one(".h-area .hz-figure")
                size = area_tag.text.strip() if area_tag else "-"

                # --- Image ---
                img_tag = item.select_one("a.hover-effect img")
                if img_tag:
                    img_url = img_tag.get("data-src") or img_tag.get("src") or "-"
                else:
                    img_url = "-"
                img_formula = f'=IMAGE("{img_url}",4,300,200)' if img_url != "-" else "-"

                date_extracted = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

                key = (title, location, price)
                if key in existing_all or key in existing_site:
                    continue

                row = [title, location, price, size, bedrooms, bathrooms,
                       img_formula, link, date_extracted, property_type]
                all_rows.append(row)

            print(f"✅ Page {page} scraped ({len(items)} listings)")
            page += 1
            time.sleep(1)

    print(f"✅ Found {len(all_rows)} new {property_type} listings in total.")
    return all_rows

# ----------------------------
# Run Scraper
# ----------------------------
all_new_rows = []

for prop_type, urls in BASE_URLS.items():
    new_rows = scrape_cornerstone(prop_type, urls, max_pages=10)
    all_new_rows.extend(new_rows)
    time.sleep(1)

# ----------------------------
# Write to Google Sheets
# ----------------------------
if all_new_rows:
    sheet_all.append_rows(all_new_rows, value_input_option='USER_ENTERED')
    sheet_site.append_rows(all_new_rows, value_input_option='USER_ENTERED')

    # Set row heights for images
    row_height_ranges = [(f"{i}:{i}", 220) for i in range(2, len(all_new_rows) + 2)]
    set_row_heights(sheet_site, row_height_ranges)

    print(f"✅ Added {len(all_new_rows)} new Cornerstone listings to Google Sheets.")
else:
    print("⚠️ No new listings found.")

print("🏁 Done! Data saved in 'RealEstateListings'.")


🔎 Scraping Sale listings from https://cornerstonerealestatesxm.com/property-type/for-sale/
✅ Page 1 scraped (11 listings)
✅ Page 2 scraped (11 listings)
✅ Page 3 scraped (11 listings)
✅ Page 4 scraped (11 listings)
✅ Page 5 scraped (11 listings)
✅ Page 6 scraped (11 listings)
✅ Page 7 scraped (11 listings)
✅ Page 8 scraped (11 listings)
✅ Page 9 scraped (11 listings)
✅ Page 10 scraped (11 listings)
✅ Found 110 new Sale listings in total.
🔎 Scraping Rent listings from https://cornerstonerealestatesxm.com/property-type/for-rent/
✅ Page 1 scraped (11 listings)
✅ Page 2 scraped (11 listings)
✅ Page 3 scraped (11 listings)
✅ Page 4 scraped (11 listings)
✅ Page 5 scraped (11 listings)
✅ Page 6 scraped (11 listings)
✅ Page 7 scraped (11 listings)
✅ Page 8 scraped (11 listings)
✅ Page 9 scraped (11 listings)
✅ Page 10 scraped (11 listings)
✅ Found 110 new Rent listings in total.
✅ Added 220 new Cornerstone listings to Google Sheets.
🏁 Done! Data saved in 'RealEstateListings'.


In [ ]:
# Ireteam 7

import requests
from bs4 import BeautifulSoup
import time
from datetime import datetime
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_formatting import set_row_heights

# ----------------------------
# Google Sheets setup
# ----------------------------
scope = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]
creds = ServiceAccountCredentials.from_json_keyfile_name("secret.json", scope)
client = gspread.authorize(creds)

SPREADSHEET_NAME = "RealEstateListings"
spreadsheet = client.open(SPREADSHEET_NAME)

def get_or_create_worksheet(name):
    for ws in spreadsheet.worksheets():
        if ws.title.strip().lower() == name.strip().lower():
            return ws
    return spreadsheet.add_worksheet(title=name, rows=2000, cols=20)

sheet_all = get_or_create_worksheet("all_listings")
sheet_site = get_or_create_worksheet("ireteam")

HEADERS = [
    "Title", "Location", "Price", "Size", "Bedrooms", "Bathrooms",
    "Image", "Link", "Date n Time Extracted", "Property Type"
]

def ensure_headers(sheet):
    existing = sheet.row_values(1)
    if existing != HEADERS:
        sheet.clear()
        sheet.append_row(HEADERS, value_input_option='USER_ENTERED')

ensure_headers(sheet_all)
ensure_headers(sheet_site)

# ----------------------------
# Duplicate Prevention
# ----------------------------
def get_existing_records(sheet):
    records = sheet.get_all_records()
    existing = set()
    for r in records:
        title = str(r.get("Title", "")).strip()
        location = str(r.get("Location", "")).strip()
        price = str(r.get("Price", "")).strip()
        if title and location and price:
            existing.add((title, location, price))
    return existing

existing_all = get_existing_records(sheet_all)
existing_site = get_existing_records(sheet_site)

# ----------------------------
# Scraper Config
# ----------------------------
BASE_URL = "https://ireteam.com/all-listing/"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
TOTAL_PAGES = 39  # fixed for now

all_rows = []

for page_num in range(1, TOTAL_PAGES + 1):
    print(f"🌍 Scraping page {page_num}...")
    url = f"{BASE_URL}?page={page_num}"
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    listings = soup.find_all("div", class_="col-md-3 bottom")
    for listing in listings:
        # --- Image ---
        img_tag = listing.find("img", class_="alignleft")
        img_url = img_tag["src"].strip() if img_tag else "-"
        img_formula = f'=IMAGE("{img_url}",4,300,200)' if img_url != "-" else "-"

        # --- Link ---
        link_tag = listing.find("a", href=True)
        link = link_tag["href"] if link_tag else "-"

        # --- Title ---
        title_tag = listing.find("div", class_="name-1")
        title = title_tag.get_text(strip=True) if title_tag else link.split("/")[-2].replace("-", " ").title()

        # --- Location ---
        loc_tag = listing.find("div", class_="tit-div")
        location = loc_tag.get_text(strip=True).replace("\n", " ") if loc_tag else "-"

        # --- Price ---
        price_tag = listing.find("div", class_="amount-btn")
        price = price_tag.get_text(" ", strip=True) if price_tag else "-"

        # --- Size / Bedrooms / Bathrooms (if available) ---
        size = "-"
        bedrooms = "-"
        bathrooms = "-"

        # --- Property Type ---
        prop_type = "Mixed"  # fixed for all listings


        # --- Date Extracted ---
        date_extracted = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        # --- Skip duplicates ---
        key = (title, location, price)
        if key in existing_all or key in existing_site:
            continue

        row = [title, location, price, size, bedrooms, bathrooms,
               img_formula, link, date_extracted, prop_type]
        all_rows.append(row)

    time.sleep(1)  # polite delay

# ----------------------------
# Write to Google Sheets
# ----------------------------
if all_rows:
    sheet_all.append_rows(all_rows, value_input_option='USER_ENTERED')
    sheet_site.append_rows(all_rows, value_input_option='USER_ENTERED')

    # Adjust row heights for images
    row_height_ranges = [(f"{i}:{i}", 220) for i in range(2, len(all_rows) + 2)]
    set_row_heights(sheet_site, row_height_ranges)

    print(f"✅ Added {len(all_rows)} new Ireteam listings to Google Sheets.")
else:
    print("⚠️ No new listings found.")

print("🏁 Done! Data saved in 'RealEstateListings'.")


🌍 Scraping page 1...
🌍 Scraping page 2...
🌍 Scraping page 3...
🌍 Scraping page 4...
🌍 Scraping page 5...
🌍 Scraping page 6...
🌍 Scraping page 7...
🌍 Scraping page 8...
🌍 Scraping page 9...
🌍 Scraping page 10...
🌍 Scraping page 11...
🌍 Scraping page 12...
🌍 Scraping page 13...
🌍 Scraping page 14...
🌍 Scraping page 15...
🌍 Scraping page 16...
🌍 Scraping page 17...
🌍 Scraping page 18...
🌍 Scraping page 19...
🌍 Scraping page 20...
🌍 Scraping page 21...
🌍 Scraping page 22...
🌍 Scraping page 23...
🌍 Scraping page 24...
🌍 Scraping page 25...
🌍 Scraping page 26...
🌍 Scraping page 27...
🌍 Scraping page 28...
🌍 Scraping page 29...
🌍 Scraping page 30...
🌍 Scraping page 31...
🌍 Scraping page 32...
🌍 Scraping page 33...
🌍 Scraping page 34...
🌍 Scraping page 35...
🌍 Scraping page 36...
🌍 Scraping page 37...
🌍 Scraping page 38...
🌍 Scraping page 39...
✅ Added 468 new Ireteam listings to Google Sheets.
🏁 Done! Data saved in 'RealEstateListings'.
